<a href="https://colab.research.google.com/github/jessica-aaao/ChordsExtractor/blob/main/ChordExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install -q -U "yt-dlp[default]"
!pip install -q -U openai-whisper
!pip install -q -U demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 42.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta

In [2]:
import requests
import whisper
import json
import pandas as pd
import os
import re
import unicodedata

In [3]:
from google.colab import drive
from IPython.display import display
from bs4 import BeautifulSoup

In [4]:
!git clone https://github.com/filipecalegario/ISMIR2019-Large-Vocabulary-Chord-Recognition.git

%cd ISMIR2019-Large-Vocabulary-Chord-Recognition
!pip install -q -r requirements.txt
%cd ..

Cloning into 'ISMIR2019-Large-Vocabulary-Chord-Recognition'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 152 (delta 51), reused 135 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 25.74 MiB | 15.63 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/ISMIR2019-Large-Vocabulary-Chord-Recognition
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
/content


In [5]:
!git clone  https://github.com/mikezzb/lyrics-sync.git

!wget -c https://repo.continuum.io/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
!chmod +x Anaconda3-2024.10-1-Linux-x86_64.sh
!bash ./Anaconda3-2024.10-1-Linux-x86_64.sh -b -f -p /usr/local

Cloning into 'lyrics-sync'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 26 (delta 1), reused 2 (delta 0), pack-reused 10 (from 1)
Receiving objects: 100% (26/26), 18.88 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (1/1), done.
--2025-03-08 02:33:57--  https://repo.continuum.io/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.177.84, 104.18.176.84, 2606:4700::6812:b054, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.177.84|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh [following]
--2025-03-08 02:33:57--  https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.a

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# !apt-get install ffmpeg
# !apt-get install espeak

# !git clone https://github.com/readbeyond/aeneas.git
# %cd aeneas
# !pip install -r requirements.txt
# !python check_dependencies.py
# %cd ..

# Common

In [8]:
class SongUrls:
    def __init__(self, name, audio, lyrics, chords):
        self.name = name
        self.audio = audio
        self.lyrics = lyrics
        self.chords = chords

    def get_name(self):
        return self.name

    def get_audio_url(self):
        return self.audio

    def get_lyrics_url(self):
        return self.lyrics

    def get_chords_url(self):
        return self.chords

In [9]:
def get_urls():
    print(f'Fetching urls...\n\n')

    file_path = '/content/drive/My Drive/TCC/CodeData/songs.csv'
    songs = pd.read_csv(file_path)

    print(f'Urls fetched!\n\n')

    return songs

In [10]:
def get_songs_from_csv():
    songs = get_urls()

    song_urls = []

    print(f'Creating SongUrls...\n\n')

    for index, row in songs.iterrows():
        song_name = slugify(row['Song Name'])
        audio_url = row['Audio URL']
        lyrics_url = row['Lyrics URL']
        chords_url = row['Chords URL']

        song_urls.append(SongUrls(song_name, audio_url, lyrics_url, chords_url))

    print(f'SongUrls created!\n\n')

    return song_urls


In [11]:
def slugify(raw_song_name):
    song_name = raw_song_name.lower()

    song_name = unicodedata.normalize('NFKD', song_name)
    song_name = song_name.encode('ascii', 'ignore').decode('ascii')

    song_name = re.sub(r'[^a-z0-9]+', '_', song_name)

    song_name = song_name.strip('_')

    return song_name

# Sound

In [12]:
def extract_voice_from_audio(audio_path, song_name):
    print(f'Extracting vocals from {audio_path}...\n\n')

    !demucs --two-stems=vocals {audio_path} -o vocals/

    return f"/content/vocals/htdemucs/{song_name}/vocals.wav"


In [23]:
def extract_audio_from_video(youtube_url, song_name):
    cookies_path = '/content/cookies.txt'
    output_path = f"/content/audios/{song_name}"

    !yt-dlp {youtube_url} --audio-format "wav" --cookies {cookies_path} -x -o {output_path}  -q

    return f"{output_path}.wav"

In [14]:
def extract_sound_recording(youtube_url, song_name):
    output_path = extract_audio_from_video(youtube_url, song_name)

    print(f'Audio saved as {output_path}!\n\n')

    return output_path

# Lyrics

## Lyrics Extraction from Webpage

In [15]:
def save_to_file(data, song_name):
    folder_path = "/content/lyrics"
    path = f"{folder_path}/{song_name}.txt"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(path, 'w') as file:
        file.write(data)

    print(f'Saved as {path}')

    return path

In [16]:
def extract_lyrics_from_html(html):
    """Extrai a letra da página HTML fornecida"""

    print(f'Fetching lyrics...!\n\n')

    lyricsTag = html.find('div', class_='lyric-original')
    lyrics = ""

    for p in lyricsTag.find_all('p'):
        for br in p.find_all('br'):
            br.replace_with('\n')
        lyrics += p.get_text() + "\n"

    print(f'Lyrics fetched!\n\n')

    return lyrics


In [17]:
def get_lyrics_from_webpage(lyric_url, song_name):
    """Obtém a página web e extrai a letra"""

    print(f'Fetching webpage {lyric_url}...\n\n')
    response = requests.get(lyric_url)

    if response.status_code == 200:
        print(f'Webpage fetched!\n\n')
        htmlContent = BeautifulSoup(response.content, 'html.parser')

        lyrics = extract_lyrics_from_html(htmlContent)
        lyrics_path = save_to_file(lyrics, song_name)

        return lyrics_path
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}\n\n")

## Lyrics Sync to Audio

### Transcription + Manual Adjustment Attempt

In [ ]:
def adjust_lyrics(transcribed_segments, comparison_lyrics):
    """Compara a letra transcrita e a letra extraída da Web, corrigindo a letra transcrita"""

    print(f"Adjusting lyrics...\n\n")

    for index, segment in enumerate(transcribed_segments):
        try:
            line = comparison_lyrics[index]

            if segment['text'].lower() != line.lower():
                segment['text'] = line

            words = line.split()
            # Testar com músicas não "especiais", mais enxutas
            if len(words) != len(segment['words']):
                print(f"Word count mismatch: {len(words)} != {len(segment['words'])}")
                continue

            for i, word in enumerate(words):
                try:
                    transcribed_word = segment['words'][i]['word']

                    if transcribed_word.lower().strip() != word.lower():
                        print(f"Word mismatch: {transcribed_word} != {word}")

                        segment['words'][i]['word'] = word
                except:
                    print(f"Word not found: {word}")
                    continue
        except:
            print(f"Line not found: {segment['text']}")
            continue

    print(f"Lyrics adjusted:\n{json.dumps(transcribed_lyrics, indent=2)}\n\n")

    return transcribed_lyrics

In [ ]:
def transcribe_audio(audio_path):
    """Transcreve o áudio usando o modelo Whisper"""

    print(f"Transcribing audio...\n\n")

    model = whisper.load_model("turbo", device='cuda')
    result = model.transcribe(audio_path, word_timestamps=True)

    print(f"Audio Transcribed!\n\n")

    return result['segments']


### Using Lyrics-Sync

In [18]:
def create_output_folder():
    output_folder = "/content/lyrics-sync/output"
    vocals_folder = output_folder + "/vocals"
    words_folder = output_folder + "/words"
    lrc_folder = output_folder + "/lrc"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(vocals_folder):
        os.makedirs(vocals_folder)

    if not os.path.exists(words_folder):
        os.makedirs(words_folder)

    if not os.path.exists(lrc_folder):
        os.makedirs(lrc_folder)

In [19]:
def get_timestamps(audio_path, lyrics_path, song_name):
    %cd /content/lyrics-sync/
    !conda env update -f environment.yml
    !source activate lsync

    from lsync import LyricsSync

    lsync = LyricsSync()
    words, lrc = lsync.sync(audio_path, lyrics_path)
    print(words)
    print(lrc)

    return f"/content/lyrics-sync/output/{song_name}.lrc"
    %cd ..

In [20]:
def get_synced_lyrics(lyric_url, audio_path, song_name):
    """Obtém a letra sincronizada com o áudio e corrigida"""

    print(f"Getting synced lyrics...\n\n")

    # transcribed_lyrics = transcribe_audio(audio_path)
    lyrics_path = get_lyrics_from_webpage(lyric_url, song_name)
    timestamps_path = get_timestamps(audio_path, lyrics_path, song_name)

    print(f"Synced lyrics ready!\n\n")

    # return (transcribed_lyrics, comparison_lyrics)
    return timestamps_path

# Chords

## Chords Extraction from Audio

In [ ]:
def extract_cipher_from(song_path, song_name):
    """Extrai a cifra, com o timestamp, a partir da música"""
    output_path = f"/content/chords/{song_name}.lab"
    !python chord_recognition.py {song_path} {output_path}

    return output_path

## Chords Sync to Lyrics

In [ ]:
def format_song_with_chords(transcribed_lyrics_path, chords_path):
    """
    Formata a LETRA_TRANSCRITA com acordes no formato tradicional de cifras.
    """
    # Ler LETRA_TRANSCRITA do JSON
    with open(transcribed_lyrics_path, 'r', encoding='utf-8') as file:
        transcribed_lyrics = json.load(file)

    # Ler ACORDES do arquivo LAB
    chords = []
    with open(chords_path, 'r', encoding='utf-8') as file:
        for line in file:
            start, end, chord = line.strip().split('\t')
            chords.append((float(start), float(end), chord))

    # Inserir acordes nos timestamps
    lyrics_with_chords = overlay_chords_on_transcribed(transcribed_lyrics, chords)
    # print(lyrics_with_chords)

    # Formatar saída com acordes e palavras
    formatted_output = format_transcribed_with_chords(lyrics_with_chords)

    return formatted_output

In [ ]:
def overlay_chords_on_transcribed(transcribed_lyrics, chords):
    """
    Associa acordes às palavras da LETRA_TRANSCRITA com base nos tempos de ACORDES.
    """
    result = []
    for segment in transcribed_lyrics.get("segments", []):
        for word_info in segment.get("words", []):
            start = word_info.get("start", None)
            end = word_info.get("end", None)
            word = word_info.get("word", "")
            chord = None

            if start is not None and end is not None:
                # Encontrar acorde correspondente ao intervalo
                for chord_start, chord_end, chord_name in chords:
                    if chord_start <= start < chord_end:
                        chord = chord_name
                        break

            result.append({
                "word": word,
                "start": start,
                "end": end,
                "chord": chord
            })
    return result

In [ ]:
def format_transcribed_with_chords(lyrics_with_chords):
    """
    Gera o formato de saída com acordes acima das palavras.
    """
    formatted_output = []
    chord_line = []
    lyrics_line = []

    for word_info in lyrics_with_chords:
        word = word_info["word"]
        chord = word_info["chord"]

        if chord:
            chord_line.append(f"{chord}")
        else:
            chord_line.append(" " * len(word))

        lyrics_line.append(word)

        # Adiciona quebra de linha para segmentos
        if word.endswith((".", ",", "?")):
            formatted_output.append(" ".join(chord_line).rstrip())
            formatted_output.append(" ".join(lyrics_line).rstrip())
            formatted_output.append("")
            chord_line = []
            lyrics_line = []

    # Adiciona o restante, se existir
    if chord_line or lyrics_line:
        formatted_output.append(" ".join(chord_line).rstrip())
        formatted_output.append(" ".join(lyrics_line).rstrip())

    return "\n".join(formatted_output)

In [ ]:
def simplify_chords_with_lyrics(chords_line, lyrics_line):
    """
    Simplifica uma linha de acordes e palavras associadas.
    Remove redundâncias e organiza a saída.
    """
    chords = chords_line.split()
    lyrics = lyrics_line.split()

    simplified_chords = []
    simplified_lyrics = []

    # Último acorde usado
    last_chord = None

    for chord, word in zip(chords, lyrics):
        if chord != last_chord:
            simplified_chords.append(chord)
            simplified_lyrics.append(word)
            last_chord = chord
        else:
            simplified_chords.append("")  # Espaço vazio na linha de acordes
            simplified_lyrics.append(word)

    # Retorna linhas formatadas
    return " ".join(simplified_chords).rstrip(), " ".join(simplified_lyrics).rstrip()

In [ ]:
def format_clean_chords_and_lyrics(raw_chords_lyrics):
    """
    Formata o texto contendo acordes redundantes e letras em um formato limpo e tradicional.
    """
    formatted_output = []
    lines = raw_chords_lyrics.strip().split("\n")

    # Processa par de linhas (chord, lyrics)
    for i in range(0, len(lines), 2):
        if i + 1 < len(lines):  # Certifique-se de que há uma linha de letras
            chords_line = lines[i]
            lyrics_line = lines[i + 1]

            # Simplificar a linha
            simplified_chords, simplified_lyrics = simplify_chords_with_lyrics(chords_line, lyrics_line)

            # Adicionar ao resultado
            formatted_output.append(simplified_chords)
            formatted_output.append(simplified_lyrics)
            formatted_output.append("")  # Linha em branco para separar blocos

    return "\n".join(formatted_output).strip()



---



# MAIN

In [21]:
# Extrai dados do csv e converte pra [SongUrls]
songs = get_songs_from_csv()

song = songs[0]
song_name = song.get_name().replace(" ", "_")
print(f"Generating \'{song_name}\' chord sheet...")

Fetching urls...


Urls fetched!


Creating SongUrls...


SongUrls created!


Generating 'lisboa' chord sheet...


In [22]:
# Baixa músicas do youtube
song_path = extract_sound_recording(song.get_audio_url(), song_name)

Audio saved as /content/audios/lisboa.wav!


Audio saved as /content/audios/lisboa.wav!




In [24]:
# Extrai letras com timestamp
create_output_folder()
lyrics_timestamped_path = get_synced_lyrics(song.get_lyrics_url(), song_path, song_name)

Getting synced lyrics...


Fetching webpage https://www.letras.mus.br/anavitoria/lisboa-part-lenine/...


Webpage fetched!


Fetching lyrics...!


Lyrics fetched!


Saved as /content/lyrics/lisboa.txt
/content/lyrics-sync
/usr/local/lib/python3.12/argparse.py:2006: FutureWarning: `remote_definition` is deprecated and will be removed in 25.9. Use `conda env create --file=URL` instead.
  action(self, namespace, argument_values, option_string)
Channels:
 - pytorch
 - nvidia
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: \ | done

pytorch-1.13.1       | 1.14 GB   | :   0% 0/1 [00:00<?, ?it/s]
libcublas-11.10.3.66 | 286.1 MB  | :   0% 0/1 [00:00<?, ?it/s]

mkl-2021.4.0         | 219.1 MB  | :   0% 0/1 [00:00<?, ?it/s]


libcusparse-11.7.4.9 | 151.1 MB  | :   0% 0/1 [00:00<?, ?it/s]



libnpp-11.7.4.75     | 129.3 MB  | :   0% 0/1 [00:00<?, ?it/s]




libcufft-10.7.2.124  | 93.6 MB   | :   0% 0/1 [00:00<?, ?it/s]





libcusolver-11.4.0.1 | 78.7 MB   | :   0% 0/1 [00:

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 178MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[<00:16.94> Eu, <00:17.21> vejo, <00:17.78> tua, <00:18.37> cara, <00:18.76> e, <00:18.90> teu, <00:19.21> querer, <00:19.78> perverso, <00:21.60> A, <00:21.80> gente, <00:22.35> fica, <00:22.94> bem, <00:23.33> aqui, <00:23.82> no, <00:24.03> chão, <00:24.33> da, <00:24.64> sala, <00:26.10> Eu, <00:26.32> te, <00:26.62> queria, <00:27.28> a, <00:27.48> vida, <00:28.06> toda,, <00:28.94> te, <00:29.21> confesso, <00:30.62> Por, <00:30.92> mim,, <00:31.28> a, <00:31.44> gente, <00:32.03> nem, <00:32.32> precisa, <00:33.16> mais, <00:33.48> da, <00:33.70> estrada, <00:35.24> Eu, <00:35.46> vejo, <00:36.03> você, <00:36.56> longe,, <00:37.14> quero, <00:37.74> você, <00:38.50> perto, <00:39.71> Fica, <00:40.30> na, <00:40.61> minha, <00:41.16> sombra,, <00:41.66> eu, <00:41.71> posso, <00:42.25> ser, <00:42.56> teu, <00:42.84> rastro, <00:44.30> Não, <00:44.61> quero, <00:45.14> tu, <00:45.42> na, <00:45.71> linha,, <00:46.32> Vivo,, <00:46.89> morto, <00:47.35> ou, <00:47.66> Claro, <00: